# Language Detection

In this exercise we try to use the previously learned methods to develop a ngram based Language Detection module.

Language Detection is the task of deciding the language of a given text using only the textual surface. 

The main assumption is that character statistics are very specific for any given language using the same alphabet.
Using language specific corpora we can establish the ngram statistic of this given language by counting all the occuring ngrams.

The assumption for the classification of new texts is that the most probable ngrams of a language are also more likely to appear in any new text.

So the algorithm is:
 - Clean the corpus in a way that it only contains (ascii) letters of the alphabet (no punctuation and digits)
 - Given a corpus of some language, count all the character ngrams and generate a list of the top `k` ngrams. This list should contain unique entries and only character ngrams occurring within tokens (no spaces).
 - Given a new text generate a list of ngrams occurring within the words. (This list does not have to be unique, because the most likely ngrams of a language are also more likely to occurr.)
 - Count how many of these ngrams are among the top k ngrams of all the `known` languages.
 - The output of the algorithm is the language with the most absolute hits.
 
 
As a formula:
Let $ngrams^k_{l_i}$ the list of
the top $k$ ngrams of language $l_i$

Let $ngrams_{input}$ be the list of ngrams of a given input text $$ L(input) = \text{argmax}_i (\{g |g \in l_i ,\text{for g in } ngrams_{input}\}) $$
 
 
Try to distinguish between English and German. 
As English corpus use the complete gutenberg corpus from nltk.
As German corpus use the tagesschau corpus we provided. (Download)


__Deliberations__
 - What are the limits of this approach?
 - Where can you see problems?
 - You could also use Zipf's law for words to classify language by checking new sentences for words in the vocabulary. What would be the difference in approach, advantages and disadvantages?

In [29]:
import nltk
import pathlib
import matplotlib.pyplot as plt

In [30]:
english_text = nltk.corpus.gutenberg.raw(nltk.corpus.gutenberg.fileids())[:1000000]

In [31]:
def load_german_text(path):
    text = ""
    for f in pathlib.Path(path).glob("*.txt"):
        with open(f, "r") as openf:
            text += openf.read()
    return text
german_text = load_german_text("../../Woche 5/tagesschau_corpus/")[:1000000]           


In [32]:
english_text[:1000]

"[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\nEmma Woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; and had lived nearly twenty-one years in the world\nwith very little to distress or vex her.\n\nShe was the youngest of the two daughters of a most affectionate,\nindulgent father; and had, in consequence of her sister's marriage,\nbeen mistress of his house from a very early period.  Her mother\nhad died too long ago for her to have more than an indistinct\nremembrance of her caresses; and her place had been supplied\nby an excellent woman as governess, who had fallen little short\nof a mother in affection.\n\nSixteen years had Miss Taylor been in Mr. Woodhouse's family,\nless as a governess than a friend, very fond of both daughters,\nbut particularly of Emma.  Between _them_ it was more the intimacy\nof sisters.  Even before Miss Taylor had ceased to hold the nominal\noffice of 

In [33]:
import string
import re
def clean(s: str) -> str:
    valid=string.ascii_letters+string.digits+" äöüß"
    s = s.lower()
    for c in valid:
        s = "".join([c if c in valid else " " for c in s ])
    s = re.sub("[\s]+", " ", s)
    return s

In [34]:
english_text = clean(english_text)
german_text = clean(german_text)

In [35]:
german_text[:100]

'david s tötete 2016 am münchner oez neun menschen aus fremdenhass oder weil er gemobbt wurde darüber'

In [36]:
n = 3
texts = {"english":english_text, "german":german_text}
def create_ngrams(texts):
    """Turn a string text into a list of character ngrams"""
    return ["".join(g) for w in nltk.tokenize.word_tokenize(texts) for g in nltk.ngrams(w, n)]
counts = {k:nltk.FreqDist(create_ngrams(v)).most_common(1000) for k, v in texts.items()}
dicts = {k:[x[0] for x in v] for k, v in counts.items()}


test = "Hallo, hier bin ich."


In [37]:
d = create_ngrams(test)
counts = {} # Counts dictionary
for language, language_dict in dicts.items():
    counts[language] = sum([tr in language_dict for tr in d]) 

print(counts)
sorted_counts = sorted(list(counts.items()), key=lambda x: x[1],reverse=True)
print(sorted_counts)
sorted_counts[0][0]


{'english': 3, 'german': 5}
[('german', 5), ('english', 3)]


'german'

In [38]:
class NGramLanguageClassifier:
    def __init__(self,texts, n=3, topk=1000):
        """ Initialize the classifier """
        self.n = n
        
        # Count ngrams
        self.counts = {k:nltk.FreqDist(self.create_ngrams(v)).most_common(topk) for k, v in texts.items()}
        
        self.dicts = {k:[x[0] for x in v] for k, v in self.counts.items()}  
        
    def create_ngrams(self, texts):
        """Turn a string text into a list of character ngrams"""
        return ["".join(g) for w in nltk.tokenize.word_tokenize(texts) for g in nltk.ngrams(w, self.n)]
    
    def classify(self,text):
        """Classify an incoming text"""
        d =  self.create_ngrams(clean(text))
        
        counts = {} # Counts dictionary
        for language, language_dict in self.dicts.items():
            counts[language] = sum([tr in language_dict for tr in d])   

        # Sort list by frequency
        sorted_counts = sorted(list(counts.items()), key=lambda x: -x[1])
        
        # Predict the language with most hits
        return sorted_counts[0][0]

In [39]:
NGLC = NGramLanguageClassifier({"english":english_text, "german":german_text}, topk=5000)  

In [40]:
NGLC.classify("Ich arbeite an der Universität Leipzig.")

'german'

In [41]:
NGLC.classify("I work at University.")

'english'

In [42]:
NGLC.classify("Ich heiße John")

'german'

In [43]:
NGLC.classify("Nachrichten über Joe Biden")

'german'

### Deliberations

Limits: 
 - Memory problems when all N-grams are stored
 - No weighting of the features
 - Dependence on the training corpus
 - possibility to represent non-valid word forms in the language by N-gram combinations

Problems:

- wrong decisions as shown in the examples

Using Words instead of N-grams:
- --> dictionary of word forms (types)
- --> more or less stop word check
- always valid word forms
- Problem: often out of vocabulary